In [8]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from keras.datasets import mnist

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
!pip install pyswarm
from pyswarm import pso

2024-04-11 07:25:06.475929: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 07:25:06.476080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 07:25:06.635899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# RPU

In [ ]:
# Define the activation function
def relu(x):
    return np.maximum(0, x)

# Define the neural network
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.random.uniform(-1, 1, (input_size, hidden_size))
        self.bias1 = np.random.uniform(-1, 1, hidden_size)
        self.weights2 = np.random.uniform(-1, 1, (hidden_size, output_size))
        self.bias2 = np.random.uniform(-1, 1, output_size)

    def predict(self, inputs):
        layer1_outputs = relu(np.dot(inputs, self.weights1) + self.bias1)
        return relu(np.dot(layer1_outputs, self.weights2) + self.bias2)

# Define the accuracy function
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

# Define the training function
def train(network, inputs, outputs, epochs=100):
    best_weights1 = network.weights1
    best_bias1 = network.bias1
    best_weights2 = network.weights2
    best_bias2 = network.bias2
    best_accuracy = 0
    random_search_count = 0
    epoch_count = 0

    while best_accuracy < 0.80:
        temp_weights1 = best_weights1 + np.random.uniform(-0.05, 0.05, best_weights1.shape)
        temp_bias1 = best_bias1 + np.random.uniform(-0.05, 0.05, best_bias1.shape)
        temp_weights2 = best_weights2 + np.random.uniform(-0.05, 0.05, best_weights2.shape)
        temp_bias2 = best_bias2 + np.random.uniform(-0.05, 0.05, best_bias2.shape)

        network.weights1 = temp_weights1
        network.bias1 = temp_bias1
        network.weights2 = temp_weights2
        network.bias2 = temp_bias2

        predictions = network.predict(inputs)
        current_accuracy = accuracy(outputs, predictions)

        random_search_count += 1
        print(f'random search {random_search_count} accuracy: {current_accuracy}')

        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_weights1 = temp_weights1
            best_bias1 = temp_bias1
            best_weights2 = temp_weights2
            best_bias2 = temp_bias2
            epoch_count += 1
            print('------------------------------')
            print(f'Epoch {epoch_count}/{epochs}, Accuracy: {current_accuracy}')
            print('------------------------------')
        else:
            network.weights1 = best_weights1
            network.bias1 = best_bias1
            network.weights2 = best_weights2
            network.bias2 = best_bias2

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], -1) / 255
x_test = x_test.reshape(x_test.shape[0], -1) / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create and train the neural network
network = NeuralNetwork(x_train.shape[1], 64, y_train.shape[1])
train(network, x_train, y_train)

# Evaluate the network
predictions = network.predict(x_test)
print(f'Test Accuracy: {accuracy(y_test, predictions)}')

In [ ]:
# Define the activation function
def relu(x):
    return np.maximum(0, x)

# Define the neural network
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.load("/kaggle/working/weights1.npy")
        self.bias1 = np.load("/kaggle/working/bias1.npy")
        self.weights2 = np.load("/kaggle/working/weights2.npy")
        self.bias2 = np.load("/kaggle/working/bias2.npy")

    def predict(self, inputs):
        layer1_outputs = relu(np.dot(inputs, self.weights1) + self.bias1)
        return relu(np.dot(layer1_outputs, self.weights2) + self.bias2)

# Define the accuracy function
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

# Define the training function
def train(network, inputs, outputs, epochs=100):
    best_weights1 = network.weights1
    best_bias1 = network.bias1
    best_weights2 = network.weights2
    best_bias2 = network.bias2
    best_accuracy = 0
    random_search_count = 0
    epoch_count = 0

    while best_accuracy < 0.80:
        temp_weights1 = best_weights1 + np.random.uniform(-0.5, 0.5, best_weights1.shape)
        temp_bias1 = best_bias1 + np.random.uniform(-0.5, 0.5, best_bias1.shape)
        temp_weights2 = best_weights2 + np.random.uniform(-0.5, 0.5, best_weights2.shape)
        temp_bias2 = best_bias2 + np.random.uniform(-0.5, 0.5, best_bias2.shape)

        network.weights1 = temp_weights1
        network.bias1 = temp_bias1
        network.weights2 = temp_weights2
        network.bias2 = temp_bias2

        predictions = network.predict(inputs)
        current_accuracy = accuracy(outputs, predictions)

        random_search_count += 1
        print(f'random search {random_search_count} accuracy: {current_accuracy}')

        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_weights1 = temp_weights1
            best_bias1 = temp_bias1
            best_weights2 = temp_weights2
            best_bias2 = temp_bias2
            epoch_count += 1
            print('------------------------------')
            print(f'Epoch {epoch_count}/{epochs}, Accuracy: {current_accuracy}')
            print('------------------------------')
        else:
            network.weights1 = best_weights1
            network.bias1 = best_bias1
            network.weights2 = best_weights2
            network.bias2 = best_bias2

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], -1) / 255
x_test = x_test.reshape(x_test.shape[0], -1) / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create and train the neural network
network1 = NeuralNetwork(x_train.shape[1], 64, y_train.shape[1])
train(network1, x_train, y_train)

# Evaluate the network
predictions = network1.predict(x_test)
print(f'Test Accuracy: {accuracy(y_test, predictions)}')

# Particle Swarm Optimization

In [13]:
# Load MNIST dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X / 255.0
y = y.astype(int)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the number of neurons in the layers
n_input = 784
n_hidden = 64
n_output = 10

# Activation function
def relu(x):
    return np.maximum(0, x)

# Output (softmax) function
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# Define the neural network
def neural_network(x, w):
    w1 = w[:n_input*n_hidden].reshape((n_input, n_hidden))
    w2 = w[n_input*n_hidden:].reshape((n_hidden, n_output))
    z1 = np.dot(x, w1)  # Hidden layer input
    a1 = relu(z1)  # Hidden layer output
    z2 = np.dot(a1, w2)  # Output layer input
    a2 = softmax(z2)  # Output layer results
    return a2


# Define the accuracy function
def accuracy(w):
    correct = 0
    for i in range(len(X_test)):
        y_pred = np.argmax(neural_network(X_test.iloc[i], w))
        if y_pred == y_test.iloc[i]:
            correct += 1
    return correct / len(X_test)



# Define the loss function (cross-entropy)
def loss_function(w):
    loss = 0
    for i in range(len(X_train)):
        y_pred = neural_network(X_train.iloc[i], w)  # Use .iloc here
        target = np.zeros(n_output)
        target[y_train.iloc[i]] = 1  # And here
        loss += -np.sum(target * np.log(y_pred))  # Cross-entropy loss
    acc = accuracy(w)
    print(f'Loss: {loss}, Accuracy: {acc}')
    return loss



# Use Particle Swarm Optimization to minimize the loss function
w_opt, loss_opt = pso(loss_function, lb=-1*np.ones((n_input+n_output)*n_hidden), ub=np.ones((n_input+n_output)*n_hidden))

print("Optimal weights:", w_opt)
print("Optimal loss:", loss_opt)

/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Loss: 1499776.8525151468, Accuracy: 0.081
Loss: 1457217.4268785822, Accuracy: 0.05878571428571429


KeyboardInterrupt: 